In [1]:
import sys
sys.path.append('methods')

from methods import Summarisation

In [2]:
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [3]:
bench = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen3-32B-Instruct-v2') 

In [4]:
print(bench.blueprint.think_pass)

 no_think


In [8]:
from utils import chunk_text
text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

81049
11


In [9]:
s1 = await bench.blueprint.run(chunks, initial_word_limit=500)

In [10]:
print(s1)

"Линия Розы: Тайны Времени и Веры" погружает читателя в масштабное эпическое путешествие, где переплетаются судьбы профессора Роберта Лэнгдона, известного специалиста по символике, и Софии Невё, наследницы древних знаний Приората Сиона, стремящихся раскрыть тайны, скрытые на протяжении веков. Сюжет разворачивается на фоне исторических и религиозных ландшафтов Парижа, Рима и швейцарских гор, ставя перед героями сложные задачи, связанные с разгадкой "Линии Розы" — символа, пронизывающего историю от Братства в Иерусалиме до современных интриг в Ватикане и Цюрихе.

В центре повествования — Роберт Лэнгдон, недавно восстановивший свою репутацию после громкого скандала, и София Невё, внучка Жака Соньера, несущая в себе наследие Приората. Их союз с Николя Соньером и капитаном Безу Фашем направлен на предотвращение захвата древних знаний епископом "Опус Деи" Мануэлем Арингаросой и монахом Сайласом, стремящимися использовать эти знания для усиления своей власти.

Действие начинается в роскошных 

In [12]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=28000, status='spawned')
Model(id='RefalMachine/RuadaptQwen3-32B-Instruct-v2', created=None, object=None, owned_by=None, max_model_len=24000, status='spawned')
Model(id='llama3-70b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned')
Model(id='DeepSeek V3', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen/Qwen2.5-VL-72B-Instruct', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen3-235B-A22B', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='deepseek-r1-32b', cr

In [13]:
from utils import chunk_text
from tqdm.asyncio import tqdm_asyncio
import time
import asyncio


async def run_benchmark(bench, total_count=100):
    final_result = []
    count = 0
    
    pbar = tqdm_asyncio(total=total_count, desc=f'Summarizing ({bench.model_name})', position=0)
    
    for item in bench.collection:
        if count >= total_count:
            print('DONE')
            break
            
        for _ in range(3):
            try:
                
                tmp = {}
                text = '\n'.join(item['text'])
                if len(text) > 800000:
                    break
                chunks = chunk_text(text)
                s = await bench.blueprint.run(chunks, initial_word_limit=500, mode='default')
                tmp['blueprint'] = s
                pbar.set_postfix({
                'blueprint': len(s),
                })
                
                s = await bench.blueprint.run(chunks, initial_word_limit=500, mode='cluster')
                tmp['blueprint_cluster'] = s
                pbar.set_postfix({
                'blueprint_cluster': len(s),
                })
                
                s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=False)
                tmp['hierarchical'] = s
                pbar.set_postfix({
                'hierarchical': len(s),
                })
                
                s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
                tmp['hierarchical_filtered'] = s
                pbar.set_postfix({
                'hierarchical_filtered': len(s),
                })
                
                final_result.append(
                    {'title': item['title'], 
                     'author': item['author'], 
                     'authors': item['authors'], 
                     'annotations': tmp, 
                     'gold_annotation': 
                     item['annotation']}
                )
                count += 1
                pbar.update(1)
                break
                
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
                
            except:
                await asyncio.sleep(10)
                continue
                
    return final_result

Summarizing:   0%|          | 0/100 [17:16<?, ?it/s]


CancelledError: 

In [ ]:
from utils import AsyncList

bench_ruadapt = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen3-32B-Instruct-v2') 
bench_deepseek = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3') 
bench_qwen = Summarisation(URL=url, KEY=key, model_name='Qwen3-235B-A22B')

models = AsyncList()

models.append(run_benchmark(bench_ruadapt))
models.append(run_benchmark(bench_deepseek))
model.append(run_benchmark(bench_qwen))

In [3]:
bench_qwen = Summarisation(URL=url, KEY=key, model_name='Qwen3-235B-A22B') 

In [ ]:
import time
from utils import chunk_text

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

model_names = ['RefalMachine/RuadaptQwen3-32B-Instruct-v2', 'DeepSeek V3', 'tpro', 'yagpt5lite']
for name in model_names:
    print(name)
    bench.change_model(key, url, name)
    a1 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'default')
        end = time.perf_counter()
        c = end - start
        print(c)
        a1 += c
    res = a1 / 3
    print(f'Blueprint: {res:.2f}')
    a2 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'cluster')
        end = time.perf_counter()
        c = end - start
        print(c)
        a2 += c
    res = a2 / 3
    print(f'Blueprint cluster: {res:.2f}')
    a3 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, False)
        end = time.perf_counter()
        c = end - start
        print(c)
        a3 += c
    res = a3 / 3
    print(f'Hierarchical: {res:.2f}')
    a4 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, True)
        end = time.perf_counter()
        c = end - start
        print(c)
        a4 += c
    res = a4 / 3
    print(f'Hierarchical filtered: {res:.2f}')
    print()
    break